In [ ]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# read measurement
csv_measurement_file = './raw_data/measurement_demo.csv'
measurement_data = pd.read_csv(csv_measurement_file, delimiter=',')
measurement_data

In [ ]:
# provide coupon information
coupon_no = 4

temp_data = measurement_data[measurement_data['No'] == coupon_no]
diameter = np.mean(temp_data.iloc[0,-3:])
area = 3.1415926*diameter*diameter/4
area

In [ ]:
# read raw data
csv_file = './raw_data/CTS-%d.csv' %coupon_no
data = pd.read_csv(csv_file, delimiter=',')
data

In [ ]:
# delete the last column
data.drop('Unnamed: 10', axis=1, inplace=True) 
data

In [ ]:
# select the loading cycles
data = data.drop(data[data['Step'] == 1].index)
data

In [ ]:
# remove the first row
data = data.iloc[1:,:]
data

In [ ]:
# balance strain and time
data['Strain(8800 (10.0.0.2 : 0):Strain 1) (%)'] = data['Strain(8800 (10.0.0.2 : 0):Strain 1) (%)'] - data['Strain(8800 (10.0.0.2 : 0):Strain 1) (%)'].iloc[0]
data['Total Time (s)'] = data['Total Time (s)'] - data['Total Time (s)'].iloc[0]
data

In [ ]:
# compute nominal strain, nominal stress, true strain, and true stress
nominal_strain = data['Strain(8800 (10.0.0.2 : 0):Strain 1) (%)']
load = data['Load(8800 (10.0.0.2 : 0):Load) (kN)']
nominal_stress = load/area*1000
true_strain = np.log(1.0 + nominal_strain/100.0)*100
true_stress = nominal_stress*(1.0 + nominal_strain/100.0)
data['True strain (%)'] = true_strain
data['True stress (MPa)'] = true_stress
data

In [ ]:
# examine the waveform of loading
plt.figure(figsize=(20,4))
plt.plot(data['Total Time (s)'], data['Strain(8800 (10.0.0.2 : 0):Strain 1) (%)'])
plt.xlim([0,10000])
plt.ylim([-5,5])

In [ ]:
# read the number of maximum cycle
max_cycle = int(data['Total Cycles'].iloc[-1])
max_cycle

In [ ]:
# hava a glance of the test data
data.drop(index = data[data['Total Cycles'] == max_cycle].index, inplace=True)

for cycle_index in range(1,max_cycle):
    num_row, num_col = 4, 5

    fig_index = (cycle_index-1)//(num_row*num_col)
    sub_plot_index = cycle_index%(num_row*num_col)
    if sub_plot_index == 0:
        sub_plot_index = num_row*num_col
    if sub_plot_index == 1:
        fig, axes = plt.subplots(num_row, num_col)
        fig.tight_layout()
        fig.subplots_adjust(wspace=0.5, hspace=0.5)
        fig.set_size_inches(16,10)

    plt.subplot(num_row, num_col, sub_plot_index)
    temp_data_0 = data[data['Total Cycles'] == cycle_index]

    for sub_cycle in [0.00, 0.25, 0.50, 0.75]:
        temp_data = temp_data_0[temp_data_0['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + sub_cycle]
        true_strain = temp_data['True strain (%)']
        true_stress = temp_data['True stress (MPa)']
        plt.plot(true_strain, true_stress)
        
    plt.legend([str(cycle_index)],
            loc='lower right',
            prop={'family' : 'Times New Roman', 'size': 12},
            frameon=False)
    plt.grid(linestyle = '--', linewidth = 1)
    
    plt.ylabel('True stress (MPa)', fontdict={'family' : 'Times New Roman', 'size': 12})
    plt.yticks(np.arange(-1000,1100,500), fontproperties = 'Times New Roman', size = 12)

    plt.xlabel('True strain (%)', fontdict={'family' : 'Times New Roman', 'size': 12})
    plt.xticks(np.arange(-6,7,2),fontproperties = 'Times New Roman', size = 12)
    
for temp_sub_plot_index in range(sub_plot_index, num_row*num_col):
    temp_row = temp_sub_plot_index//num_col
    temp_col = temp_sub_plot_index%num_col
    fig.delaxes(axes[temp_row][temp_col])

In [ ]:
# compute peak stress, elastic modulus, elastic radius, and back stress of each cycle
data['Peak stress (MPa)'] = 0.0
data['Elastic modulus (N/mm2)'] = 0.0
data['Elastic radius (MPa)'] = 0.0
data['Back stress (MPa)'] = 0.0

file_path = './data_processing'
if not os.path.exists(file_path):
    os.mkdir(file_path)

saving_file_path = './data_processing/CTS-%d/' %coupon_no
if not os.path.exists(saving_file_path):
    os.mkdir(saving_file_path)


for cycle_index in range(1,max_cycle):
    num_row, num_col = 4, 5

    fig_index = (cycle_index-1)//(num_row*num_col)
    sub_plot_index = cycle_index%(num_row*num_col)
    if sub_plot_index == 0:
        sub_plot_index = num_row*num_col
    if sub_plot_index == 1:
        fig, axes = plt.subplots(num_row, num_col)
        fig.tight_layout()
        fig.subplots_adjust(wspace=0.5, hspace=0.5)
        fig.set_size_inches(16,10)

    plt.subplot(num_row, num_col, sub_plot_index)
    temp_data = data[data['Total Cycles'] == cycle_index]
    true_strain = temp_data['True strain (%)']
    true_stress = temp_data['True stress (MPa)']
    plt.plot(true_strain, true_stress)

    if cycle_index == 1:
        temp_data = data[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.00]
        true_strain = np.array(temp_data['True strain (%)'])
        true_stress = np.array(temp_data['True stress (MPa)'])
        max_stress = np.max(temp_data['True stress (MPa)'])
        linear_temp_data = temp_data[(temp_data['True strain (%)'] > 0.0001) & (temp_data['True strain (%)'] < 0.003)]
        linear_true_strain = np.array(linear_temp_data['True strain (%)'])
        linear_true_stress = np.array(linear_temp_data['True stress (MPa)'])
        linear_true_strain_array = np.vstack([linear_true_strain, np.ones(len(linear_true_strain))]).T
        # linear_true_strain
        m, c = np.linalg.lstsq(linear_true_strain_array,linear_true_stress,rcond=None)[0]
        plt.plot(true_strain, m*true_strain + c, 'r', linestyle = '--', label='Fitted line')
        
        data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.00,'Peak stress (MPa)'] = max_stress
        data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.00,'Elastic modulus (N/mm2)'] = m*100
        data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.00,'Elastic radius (MPa)'] = max_stress
        data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.00,'Back stress (MPa)'] = 0.0
    else:
        data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.00,'Peak stress (MPa)'] = min_stress
        data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.00,'Elastic modulus (N/mm2)'] = m*100
        data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.00,'Elastic radius (MPa)'] = elastic_radius
        data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.00,'Back stress (MPa)'] = back_stress
    
    # maximum turning point
    temp_data = data[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.25]
    true_strain = np.array(temp_data['True strain (%)'])
    true_stress = np.array(temp_data['True stress (MPa)'])
    max_stress = np.max(temp_data['True stress (MPa)'])
    max_strain = temp_data['True strain (%)'].loc[temp_data['True stress (MPa)'] == max_stress]
    linear_temp_data = temp_data[(temp_data['True stress (MPa)'] > 0.2*max_stress) & (temp_data['True stress (MPa)'] < 0.9*max_stress)]
    linear_true_strain = np.array(linear_temp_data['True strain (%)'])
    linear_true_stress = np.array(linear_temp_data['True stress (MPa)'])
    linear_true_strain_array = np.vstack([linear_true_strain, np.ones(len(linear_true_strain))]).T
    # linear_true_strain
    m, c = np.linalg.lstsq(linear_true_strain_array,linear_true_stress,rcond=None)[0]
    true_stress_fit = m*temp_data['True strain (%)'] + c
    plt.plot(temp_data['True strain (%)'], true_stress_fit, 'r', linestyle = '--', label='Fitted line')

    diff_tol = -15
    temp_data_0 = temp_data[(temp_data['True stress (MPa)'] <= 0.9*max_stress)]
    true_stress_fit = m*temp_data_0['True strain (%)'] + c
    linear_stress_diff = np.abs(true_stress_fit - temp_data_0['True stress (MPa)'] - diff_tol) 
    min_diff = np.min(linear_stress_diff)
    linear_strain = temp_data_0['True strain (%)'].loc[linear_stress_diff == min_diff]
    linear_stress = temp_data_0['True stress (MPa)'].loc[linear_stress_diff == min_diff]
    elastic_radius = 0.5*(max_stress - linear_stress).iloc[0]
    back_stress = 0.5*(max_stress + linear_stress).iloc[0]
    plt.plot(linear_strain, linear_stress, 'ro')
    plt.plot(max_strain, max_stress, 'ro')
    
    data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.25,'Peak stress (MPa)'] = max_stress
    data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.50,'Peak stress (MPa)'] = max_stress
    data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.25,'Elastic modulus (N/mm2)'] = m*100
    data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.50,'Elastic modulus (N/mm2)'] = m*100
    data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.25,'Elastic radius (MPa)'] = elastic_radius
    data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.50,'Elastic radius (MPa)'] = elastic_radius
    data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.25,'Back stress (MPa)'] = back_stress
    data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.50,'Back stress (MPa)'] = back_stress

    # minimum turning point
    temp_data = data[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.75]
    true_strain = np.array(temp_data['True strain (%)'])
    true_stress = np.array(temp_data['True stress (MPa)'])
    min_stress = np.min(temp_data['True stress (MPa)'])
    min_strain = temp_data['True strain (%)'].loc[temp_data['True stress (MPa)'] == min_stress]
    linear_temp_data = temp_data[(temp_data['True stress (MPa)'] < 0.1*min_stress) & (temp_data['True stress (MPa)'] > 0.9*min_stress)]
    linear_true_strain = np.array(linear_temp_data['True strain (%)'])
    linear_true_stress = np.array(linear_temp_data['True stress (MPa)'])
    linear_true_strain_array = np.vstack([linear_true_strain, np.ones(len(linear_true_strain))]).T
    # linear_true_strain
    m, c = np.linalg.lstsq(linear_true_strain_array,linear_true_stress,rcond=None)[0]
    true_stress_fit = m*temp_data['True strain (%)'] + c
    plt.plot(temp_data['True strain (%)'], true_stress_fit, 'r', linestyle = '--', label='Fitted line')

    temp_data_0 = temp_data[(temp_data['True stress (MPa)'] >= 0.9*min_stress)]
    true_stress_fit = m*temp_data_0['True strain (%)'] + c
    linear_stress_diff = np.abs(temp_data_0['True stress (MPa)'] - true_stress_fit - diff_tol) 
    min_diff = np.min(linear_stress_diff)
    linear_strain = temp_data_0['True strain (%)'].loc[linear_stress_diff == min_diff]
    linear_stress = temp_data_0['True stress (MPa)'].loc[linear_stress_diff == min_diff]
    elastic_radius = -0.5*(min_stress - linear_stress).iloc[0]
    back_stress =  0.5*(min_stress + linear_stress).iloc[0]
    plt.plot(linear_strain, linear_stress, 'ro')
    plt.plot(min_strain, min_stress, 'ro')

    data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.75,'Peak stress (MPa)'] = min_stress
    data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.75,'Elastic modulus (N/mm2)'] = m*100
    data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.75,'Elastic radius (MPa)'] = elastic_radius
    data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.75,'Back stress (MPa)'] = back_stress
        
    plt.legend([str(cycle_index)],
            loc='lower right',
            prop={'family' : 'Times New Roman', 'size': 12},
            frameon=False)
    plt.grid(linestyle = '--', linewidth = 1)
    
    plt.ylabel('True stress (MPa)', fontdict={'family' : 'Times New Roman', 'size': 12})
    plt.yticks(np.arange(-1000,1100,500), fontproperties = 'Times New Roman', size = 12)
    plt.ylim([-1000,1000])

    plt.xlabel('True strain (%)', fontdict={'family' : 'Times New Roman', 'size': 12})
    plt.xticks(np.arange(-6,7,2),fontproperties = 'Times New Roman', size = 12)
    plt.xlim([-6,6])

    if cycle_index == (max_cycle - 1):
        data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index + 0.00,'Elastic modulus (N/mm2)'] = m*100
        data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index + 0.00,'Elastic radius (MPa)'] = elastic_radius
        data.loc[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index + 0.00,'Back stress (MPa)'] = back_stress

    if (sub_plot_index == num_row*num_col):
        saving_file_name = 'Q690_CTS_%d_individual_cycle_analysis_%d' %(coupon_no,fig_index+1)
        fig.savefig(saving_file_path + '/' + saving_file_name + '.png', dpi=300, bbox_inches = 'tight')
        fig.savefig(saving_file_path + '/' + saving_file_name + '.pdf', bbox_inches = 'tight')


if temp_sub_plot_index < num_row*num_col:
    for temp_sub_plot_index in range(sub_plot_index, num_row*num_col):
        temp_row = temp_sub_plot_index//num_col
        temp_col = temp_sub_plot_index%num_col
        fig.delaxes(axes[temp_row][temp_col])

if cycle_index == (max_cycle-1):
    saving_file_name = 'Q690_CTS_%d_individual_cycle_analysis_%d' %(coupon_no,fig_index+1)
    fig.savefig(saving_file_path + '/' + saving_file_name + '.png', dpi=300, bbox_inches = 'tight')
    fig.savefig(saving_file_path + '/' + saving_file_name + '.pdf', bbox_inches = 'tight')

In [ ]:
data

In [ ]:
# compute plastic strain
data['Plastic strain (%)'] = data['True strain (%)'] - data['True stress (MPa)']/data['Elastic modulus (N/mm2)']
data

In [ ]:
# compute equivalent plastic strain
data['Equivalent plastic strain'] = 0.0
temp = data['Equivalent plastic strain']
data['Equivalent plastic strain'].iloc[1:] = data['Plastic strain (%)'].shift(1)
data['Equivalent plastic strain'] = np.abs(data['Plastic strain (%)'] - data['Equivalent plastic strain']).cumsum()
data['Equivalent plastic strain'] = data['Equivalent plastic strain']/100
data

In [ ]:
csv_file_after_processing = 'Q690_CTS_%d_data_processing' %coupon_no
data.to_csv(saving_file_path + '/' + csv_file_after_processing + '.csv', encoding='utf-8-sig')

In [ ]:
# save peak data 
for cycle_index in range(1,max_cycle):

    if cycle_index == 1:
        max_stress = np.max(data[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.00]['True stress (MPa)'])
        peak_data = data[data['True stress (MPa)'] == max_stress].iloc[:1,:]
        peak_data = peak_data.append(data[data['True stress (MPa)'] == max_stress].iloc[:1,:])
    
    # maximum turning point
    peak_data = peak_data.append(data[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.25].iloc[:1,:])

    # minimum turning point
    peak_data = peak_data.append(data[data['Total Cycle Count(8800 (10.0.0.2 : 0) Waveform)'] == cycle_index - 1 + 0.75].iloc[:1,:])


peak_data_after_processing = 'Q690_CTS_%d_peak_data' %coupon_no
peak_data.to_csv(saving_file_path + '/' + peak_data_after_processing + '.csv', encoding='utf-8-sig')

peak_data
    

In [ ]:
# save in tension and compression separately
temp_peak_data = peak_data.reset_index()
temp_data_tension = temp_peak_data.iloc[lambda x: x.index % 2 == 0]
temp_data_compression = temp_peak_data.iloc[lambda x: x.index % 2 == 1]


peak_data_tension = 'Q690_CTS_%d_peak_data_tension' %coupon_no
peak_data_compression = 'Q690_CTS_%d_peak_data_compression' %coupon_no
temp_data_tension.to_csv(saving_file_path + '/' + peak_data_tension + '.csv', encoding='utf-8-sig')
temp_data_compression.to_csv(saving_file_path + '/' + peak_data_compression + '.csv', encoding='utf-8-sig')

In [ ]:
fig = plt.figure(figsize=(15,5))
fig.tight_layout()
temp_peak_data = peak_data.reset_index()
temp_data = temp_peak_data.iloc[lambda x: x.index % 2 == 0]
plt.plot(temp_data['Equivalent plastic strain'], temp_data['Peak stress (MPa)'], linewidth = 2.5)
plt.plot(temp_data['Equivalent plastic strain'], temp_data['Elastic radius (MPa)'], linewidth = 2.5)
plt.plot(temp_data['Equivalent plastic strain'], np.abs(temp_data['Back stress (MPa)']), 'r--', linewidth = 2.5)
plt.xlim([0,13])

plt.legend(['$\\sigma$','Y', '$\\alpha$'],
            loc='upper right',
            prop={'family' : 'Times New Roman', 'size': 12},
            frameon=False)
# plt.grid(linestyle = '--', linewidth = 1)

plt.ylabel('True stress (MPa)', fontdict={'family' : 'Times New Roman', 'size': 12})
plt.yticks(np.arange(0,1000,200), fontproperties = 'Times New Roman', size = 12)
# plt.ylim([-500,500])

plt.xlabel('Equivalent plastic strain', fontdict={'family' : 'Times New Roman', 'size': 12})
plt.xticks(np.arange(0,13,1),fontproperties = 'Times New Roman', size = 12)
# plt.xlim([-6,6])
